In [ ]:
# prompt: create pip command to install intake and intake-esm

!pip install intake intake-esm xarray netCDF4

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
xr.set_options(display_style='html')
import intake
%matplotlib inline

In [ ]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)
col

,unique
activity_id,18
institution_id,36
source_id,88
experiment_id,170
member_id,657
table_id,37
variable_id,700
grid_label,10
zstore,514818
dcpp_init_year,60


In [ ]:
# cat = col.search(experiment_id=['historical', 'ssp245'], table_id='Amon', variable_id='tas')
# cat.df

# 1. Build your catalog as usual:
cat = col.search(
    experiment_id=['historical','ssp245'],
    table_id='Amon',
    variable_id='tas',
    member_id='.*f1$',       # re‐use your previous regex
    #grid_label='^gr$'
)

cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,NOAA-GFDL,GFDL-ESM4,historical,r3i1p1f1,Amon,tas,gr1,gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/hist...,NaN,20180701
1,CMIP,NOAA-GFDL,GFDL-ESM4,historical,r2i1p1f1,Amon,tas,gr1,gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/hist...,NaN,20180701
2,ScenarioMIP,NOAA-GFDL,GFDL-CM4,ssp245,r1i1p1f1,Amon,tas,gr1,gs://cmip6/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-CM...,NaN,20180701
3,CMIP,NOAA-GFDL,GFDL-CM4,historical,r1i1p1f1,Amon,tas,gr1,gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/histo...,NaN,20180701
4,ScenarioMIP,NOAA-GFDL,GFDL-ESM4,ssp245,r3i1p1f1,Amon,tas,gr1,gs://cmip6/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ES...,NaN,20180701
...,...,...,...,...,...,...,...,...,...,...,...
808,ScenarioMIP,MIROC,MIROC6,ssp245,r50i1p1f1,Amon,tas,gn,gs://cmip6/CMIP6/ScenarioMIP/MIROC/MIROC6/ssp2...,NaN,20210917
809,ScenarioMIP,MIROC,MIROC6,ssp245,r29i1p1f1,Amon,tas,gn,gs://cmip6/CMIP6/ScenarioMIP/MIROC/MIROC6/ssp2...,NaN,20210917
810,CMIP,CMCC,CMCC-CM2-SR5,historical,r3i1p2f1,Amon,tas,gn,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-SR5/histor...,NaN,20211108
811,CMIP,CMCC,CMCC-CM2-SR5,historical,r2i1p2f1,Amon,tas,gn,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-SR5/histor...,NaN,20211109


In [ ]:
# dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True}, skip_on_error=True)
# list(dset_dict.keys())

dset_dict = cat.to_dataset_dict(
    xarray_open_kwargs={"consolidated": True},
    storage_options={"token": "anon"},
    skip_on_error=True
)

list(dset_dict.keys())


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


/usr/local/lib/python3.11/dist-packages/dask/array/core.py:4996: PerformanceWarning: Increasing number of chunks by factor of 16
  result = blockwise(
/usr/local/lib/python3.11/dist-packages/dask/array/core.py:4996: PerformanceWarning: Increasing number of chunks by factor of 16
  result = blockwise(


['ScenarioMIP.AS-RCEC.TaiESM1.ssp245.Amon.gn',
 'CMIP.SNU.SAM0-UNICON.historical.Amon.gn',
 'CMIP.CSIRO-ARCCSS.ACCESS-CM2.historical.Amon.gn',
 'CMIP.NASA-GISS.GISS-E2-1-H.historical.Amon.gn',
 'ScenarioMIP.NIMS-KMA.KACE-1-0-G.ssp245.Amon.gr',
 'CMIP.NCAR.CESM2-FV2.historical.Amon.gn',
 'ScenarioMIP.FIO-QLNM.FIO-ESM-2-0.ssp245.Amon.gn',
 'ScenarioMIP.THU.CIESM.ssp245.Amon.gr',
 'ScenarioMIP.NCC.NorESM2-MM.ssp245.Amon.gn',
 'CMIP.EC-Earth-Consortium.EC-Earth3-Veg-LR.historical.Amon.gr',
 'ScenarioMIP.EC-Earth-Consortium.EC-Earth3-CC.ssp245.Amon.gr',
 'CMIP.INM.INM-CM5-0.historical.Amon.gr1',
 'CMIP.EC-Earth-Consortium.EC-Earth3-Veg.historical.Amon.gr',
 'CMIP.AS-RCEC.TaiESM1.historical.Amon.gn',
 'CMIP.CCCma.CanESM5.historical.Amon.gn',
 'CMIP.NOAA-GFDL.GFDL-CM4.historical.Amon.gr1',
 'CMIP.IPSL.IPSL-CM6A-LR-INCA.historical.Amon.gr',
 'CMIP.NIMS-KMA.KACE-1-0-G.historical.Amon.gr',
 'CMIP.IPSL.IPSL-CM6A-LR.historical.Amon.gr',
 'ScenarioMIP.NOAA-GFDL.GFDL-ESM4.ssp245.Amon.gr1',
 'CMIP.UA

# Task
Generate Python code to process climate model data. The code should iterate through a dictionary of xarray Datasets (`dset_dict`), where keys are model names and values are Datasets containing 'historical' and 'ssp245' scenarios with multiple ensemble members. For each model and each ensemble member within that model, concatenate the 'historical' and 'ssp245' data along the time dimension. Then, calculate the global average and the annual average of the 'tas' variable for each concatenated ensemble member. Store these annual global averages in a pandas DataFrame where the columns are named as 'model_name_ensemble_member' and the index is the year. Ensure that dask is used for lazy loading and computation to avoid memory issues. Skip any models that cause errors during processing.

In [ ]:
# Dictionary to hold concatenated datasets for each model and member
concatenated_datasets = {}
cnt = 0

for key, ds in dset_dict.items():
    # Extract model name, experiment, and member from the key
    key_parts = key.split('.')
    if len(key_parts) >= 6: # Ensure there are enough parts to extract info
        activity_id, institution_id, source_id, experiment_id, table_id, grid_label = key_parts[:6]
        model_name = f"{institution_id}.{source_id}.{grid_label}" # Construct a unique model identifier

        # Check if 'member_id' dimension exists
        if 'member_id' in ds.dims:
             for member_id in ds['member_id'].values:
                # Create a unique key for each model and member
                model_member_key = f"{model_name}_{member_id}"

                # Select the data for the current experiment and member
                ds_selected = ds.sel(member_id=member_id)

                # Store the selected dataset based on experiment and model_member_key
                if model_member_key not in concatenated_datasets:
                    concatenated_datasets[model_member_key] = {}

                concatenated_datasets[model_member_key][experiment_id] = ds_selected
        else:
             print(f"Skipping {key}: 'member_id' dimension not found.")
    else:
        print(f"Skipping {key}: Key format is not as expected.")


annual_global_averages = {}

for model_member_key, experiments_dict in concatenated_datasets.items():
    try:
        # Check if both historical and ssp245 experiments are present for the member
        if 'historical' in experiments_dict and 'ssp245' in experiments_dict:
            ds_historical = experiments_dict['historical']
            ds_ssp245 = experiments_dict['ssp245']

            # Concatenate along the time dimension
            ds_concatenated = xr.concat([ds_historical, ds_ssp245], dim='time')

            # Decode times after concatenation
            ds_concatenated = xr.decode_cf(ds_concatenated)

            # Calculate global average of 'tas', handling potential errors
            if 'tas' in ds_concatenated.variables and 'lat' in ds_concatenated.coords and 'lon' in ds_concatenated.coords:
                # Check if 'lat' is a DataArray before calling .cos()
                if isinstance(ds_concatenated['lat'], xr.DataArray):
                    weights = np.cos(np.deg2rad(ds_concatenated['lat']))
                    # Ensure weights and data have compatible dimensions before applying weights
                    if weights.dims == ds_concatenated['tas'].dims[:2]: # Check if weights match spatial dims
                         global_avg_tas = ds_concatenated['tas'].weighted(weights).mean(('lat', 'lon'))
                    else:
                        global_avg_tas = ds_concatenated['tas'].mean(('lat', 'lon')) # Fallback to simple mean
                else:
                    print(f"Skipping weighted global averaging for {model_member_key}: 'lat' is not a DataArray.")
                    global_avg_tas = ds_concatenated['tas'].mean(('lat', 'lon')) # Fallback to simple mean


                # Calculate annual average of the globally averaged 'tas'
                if 'time' in global_avg_tas.coords:
                    annual_global_avg_tas = global_avg_tas.resample(time='YE').mean()
                    annual_global_avg_tas = annual_global_avg_tas.squeeze()
                    # Store the annual global average
                    annual_global_averages[model_member_key] = annual_global_avg_tas
                    print(f"done: {model_member_key}")
                    cnt += 1
                    # if cnt > 10: break
                else:
                    print(f"Skipping annual averaging for {model_member_key}: No time coordinate found after global averaging.")
            else:
                print(f"Skipping global averaging for {model_member_key}: Missing 'tas' variable or lat/lon coordinates.")

        else:
             print(f"Skipping {model_member_key}: Both historical and ssp245 experiments not found for this member.")

    except Exception as e:
        print(f"Skipping processing for {model_member_key} due to error: {e}")

print(f"Number of annual global average time series generated: {len(annual_global_averages)}")

# Convert the dictionary of annual global averages to a pandas DataFrame
annual_global_averages_df = pd.DataFrame()

if annual_global_averages: # Check if the dictionary is not empty
    for key, da in annual_global_averages.items():
        try:
            # Ensure the time coordinate is decoded and convert to datetime for the DataFrame index
            if 'time' in da.coords:
                # Convert to pandas Series for easy conversion to DataFrame
                series = da.to_series()
                annual_global_averages_df[key] = series.values
                # Use the year from the time index for the DataFrame index
                annual_global_averages_df.index = series.index.year
            else:
                 print(f"Skipping DataFrame conversion for {key}: No time coordinate found in DataArray.")

        except Exception as e:
            print(f"Skipping DataFrame conversion for {key} due to error: {e}")

# Sort the DataFrame by index (year)
if not annual_global_averages_df.empty:
    annual_global_averages_df = annual_global_averages_df.sort_index()


display(annual_global_averages_df.head())
display(annual_global_averages_df.tail())

done: AS-RCEC.TaiESM1.gn_r1i1p1f1
Skipping SNU.SAM0-UNICON.gn_r1i1p1f1: Both historical and ssp245 experiments not found for this member.
done: CSIRO-ARCCSS.ACCESS-CM2.gn_r1i1p1f1
done: CSIRO-ARCCSS.ACCESS-CM2.gn_r2i1p1f1
done: CSIRO-ARCCSS.ACCESS-CM2.gn_r3i1p1f1
done: CSIRO-ARCCSS.ACCESS-CM2.gn_r4i1p1f1
done: CSIRO-ARCCSS.ACCESS-CM2.gn_r5i1p1f1
Skipping NASA-GISS.GISS-E2-1-H.gn_r10i1p1f1: Both historical and ssp245 experiments not found for this member.
Skipping NASA-GISS.GISS-E2-1-H.gn_r1i1p1f1: Both historical and ssp245 experiments not found for this member.
done: NASA-GISS.GISS-E2-1-H.gn_r1i1p3f1
Skipping NASA-GISS.GISS-E2-1-H.gn_r1i1p5f1: Both historical and ssp245 experiments not found for this member.
Skipping NASA-GISS.GISS-E2-1-H.gn_r2i1p1f1: Both historical and ssp245 experiments not found for this member.
done: NASA-GISS.GISS-E2-1-H.gn_r2i1p3f1
Skipping NASA-GISS.GISS-E2-1-H.gn_r2i1p5f1: Both historical and ssp245 experiments not found for this member.
Skipping NASA-GISS.GI

<ipython-input-21-b372412aa01f>:94: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  annual_global_averages_df[key] = series.values
<ipython-input-21-b372412aa01f>:94: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  annual_global_averages_df[key] = series.values
<ipython-input-21-b372412aa01f>:94: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

Skipping DataFrame conversion for E3SM-Project.E3SM-1-1.gr_r1i1p1f1 due to error: Length of values (175) does not match length of index (251)


<ipython-input-21-b372412aa01f>:94: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  annual_global_averages_df[key] = series.values
<ipython-input-21-b372412aa01f>:94: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  annual_global_averages_df[key] = series.values
<ipython-input-21-b372412aa01f>:94: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

,AS-RCEC.TaiESM1.gn_r1i1p1f1,CSIRO-ARCCSS.ACCESS-CM2.gn_r1i1p1f1,CSIRO-ARCCSS.ACCESS-CM2.gn_r2i1p1f1,CSIRO-ARCCSS.ACCESS-CM2.gn_r3i1p1f1,CSIRO-ARCCSS.ACCESS-CM2.gn_r4i1p1f1,CSIRO-ARCCSS.ACCESS-CM2.gn_r5i1p1f1,NASA-GISS.GISS-E2-1-H.gn_r1i1p3f1,NASA-GISS.GISS-E2-1-H.gn_r2i1p3f1,NASA-GISS.GISS-E2-1-H.gn_r3i1p3f1,NASA-GISS.GISS-E2-1-H.gn_r4i1p3f1,...,MIROC.MIROC6.gn_r4i1p1f1,MIROC.MIROC6.gn_r50i1p1f1,MIROC.MIROC6.gn_r5i1p1f1,MIROC.MIROC6.gn_r6i1p1f1,MIROC.MIROC6.gn_r7i1p1f1,MIROC.MIROC6.gn_r8i1p1f1,MIROC.MIROC6.gn_r9i1p1f1,NCAR.CESM2.gn_r10i1p1f1,NCAR.CESM2.gn_r11i1p1f1,NCAR.CESM2.gn_r4i1p1f1
1850,276.775024,277.624634,277.538300,277.380890,277.595490,277.744080,278.264404,278.193268,278.086670,278.171509,...,280.156830,279.723114,279.847992,279.727539,279.400024,279.926910,279.783813,277.351196,277.614258,277.332214
1851,277.088959,277.637054,277.523895,277.429291,277.658600,277.581665,278.247803,278.439056,277.929108,278.271881,...,280.147491,279.850433,279.720428,279.806793,279.650513,279.801941,279.673218,277.453979,277.677917,277.321136
1852,277.082062,277.690948,277.180695,277.439636,277.574799,277.614410,278.173584,278.611389,278.240417,278.222565,...,279.859467,280.125702,279.782074,279.659088,279.810852,279.779724,279.691162,277.328094,277.795074,277.503754
1853,276.975250,277.545166,277.551422,277.429108,277.596344,277.529999,278.164459,278.337952,278.179962,278.404388,...,279.691132,279.851379,279.761230,279.896149,279.634613,279.593597,279.831818,277.301361,277.670532,277.357483
1854,276.995728,277.521149,277.477020,277.529694,277.426331,277.755310,278.230988,278.177124,277.996124,278.305603,...,279.704926,279.666962,279.886414,279.803467,279.616669,279.663269,279.950470,277.289093,277.727966,277.382782


,AS-RCEC.TaiESM1.gn_r1i1p1f1,CSIRO-ARCCSS.ACCESS-CM2.gn_r1i1p1f1,CSIRO-ARCCSS.ACCESS-CM2.gn_r2i1p1f1,CSIRO-ARCCSS.ACCESS-CM2.gn_r3i1p1f1,CSIRO-ARCCSS.ACCESS-CM2.gn_r4i1p1f1,CSIRO-ARCCSS.ACCESS-CM2.gn_r5i1p1f1,NASA-GISS.GISS-E2-1-H.gn_r1i1p3f1,NASA-GISS.GISS-E2-1-H.gn_r2i1p3f1,NASA-GISS.GISS-E2-1-H.gn_r3i1p3f1,NASA-GISS.GISS-E2-1-H.gn_r4i1p3f1,...,MIROC.MIROC6.gn_r4i1p1f1,MIROC.MIROC6.gn_r50i1p1f1,MIROC.MIROC6.gn_r5i1p1f1,MIROC.MIROC6.gn_r6i1p1f1,MIROC.MIROC6.gn_r7i1p1f1,MIROC.MIROC6.gn_r8i1p1f1,MIROC.MIROC6.gn_r9i1p1f1,NCAR.CESM2.gn_r10i1p1f1,NCAR.CESM2.gn_r11i1p1f1,NCAR.CESM2.gn_r4i1p1f1
2096,281.989777,281.803070,281.668152,281.602386,281.931366,281.948181,282.333588,282.049835,282.311676,282.228516,...,NaN,282.492706,NaN,NaN,NaN,NaN,NaN,281.791901,281.628326,281.506256
2097,281.974030,281.608002,281.723480,281.665131,281.937653,281.852264,282.248199,282.161865,282.192139,281.785187,...,NaN,282.504730,NaN,NaN,NaN,NaN,NaN,281.670624,281.645660,281.704010
2098,281.995575,281.703949,281.557800,281.684021,282.033417,281.777069,282.072876,281.908875,282.376892,281.873749,...,NaN,282.428925,NaN,NaN,NaN,NaN,NaN,281.352844,281.625977,281.711395
2099,281.742828,281.674927,281.634613,281.822784,282.074738,281.744751,282.301178,282.037994,282.323975,282.144409,...,NaN,282.639191,NaN,NaN,NaN,NaN,NaN,281.563446,281.705536,281.478485
2100,281.747223,281.653168,281.643158,281.820465,282.115021,281.858582,282.398315,282.207123,282.169373,282.447113,...,NaN,282.800293,NaN,NaN,NaN,NaN,NaN,282.036041,281.644897,281.798462


In [ ]:
# Save the DataFrame to a CSV file
annual_global_averages_df.to_csv('annual_global_averages.csv', index=True, header=True)

print("DataFrame saved to annual_global_averages.csv")

DataFrame saved to annual_global_averages.csv


In [ ]:
len(annual_global_averages_df.columns)

183